In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("0506_ENG_Korean_plus_Sentiment.csv")

In [5]:
# 0 긍, 1 부
df[:3]

,reviews,title,K_words_list,k_food,place,k_slang,k_fashion_beauty,k_pop,history_event,k_culture,pos_neg
0,how criminal profiling came to be in south kor...,악의마음을읽는자들,"['raincoat killer', 'ost', 'korea', 'south kor...",0.0,3.0,0.0,0.0,1.0,1.0,0.0,0
1,g r i p p i n g all the cases are based on tru...,악의마음을읽는자들,"['slow burn', 'ost', 'kwon il yong', 'korea', ...",0.0,2.0,0.0,0.0,1.0,1.0,2.0,0
2,darkness cannot drive out darkness only light ...,악의마음을읽는자들,"['fighting', 'kwon il yong', 'korea', 'korean'...",0.0,2.0,0.0,0.0,1.0,1.0,2.0,0


In [10]:
pos_df = df[df.pos_neg == 0]
neg_df = df[df.pos_neg == 1]

In [9]:
pos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18991 entries, 0 to 27697
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   reviews           18991 non-null  object 
 1   title             18991 non-null  object 
 2   K_words_list      18991 non-null  object 
 3   k_food            18991 non-null  float64
 4   place             18991 non-null  float64
 5   k_slang           18991 non-null  float64
 6   k_fashion_beauty  18991 non-null  float64
 7   k_pop             18991 non-null  float64
 8   history_event     18991 non-null  float64
 9   k_culture         18991 non-null  float64
 10  pos_neg           18991 non-null  int64  
dtypes: float64(7), int64(1), object(3)
memory usage: 1.7+ MB


In [12]:
pos_df.reviews[0]

"how criminal profiling came to be in south korea he who fights with monsters should look to it that he himself does not become a monster if you gaze long into an abyss the abyss also gazes into you this drama is based on a nonfiction book written by south korea's first criminal profiler given this most of the cases all except one in which i am unsure of are based on true cases in s korea the storyline is straightforward and goes in chronological order like a documentary and has several time jumps to show the years passed there were a handful of slow moments but with only episodes they weren't too bad i overlooked them for the tight story development superb acting and perfect ost and bgm some people complain that it's too slow but this is based on true events the writer of this drama can only dramaticize it so far i'm glad they stick to the facts to stay true to the true cases while adding some thrill suspense in catching the killer while the characters don't go deep into psychological

In [2]:
drama_titles = df.groupby('title',as_index=False).count().title
drama_titles

0                          18어게인
1                  365:운명을거스르는1년
2                       60일지정생존자
3                           D.P.
4     검색어를입력하세요WWW
                 ...            
87                  하늘에서내리는일억개의별
88                          하이에나
89                      한번다녀왔습니다
90                          해피니스
91                         호텔델루나
Name: title, Length: 92, dtype: object

In [14]:
reviews = pos_df.reviews.to_list()


sentences = ",".join(reviews)
reviews_by_drama = []
for title in drama_titles:
    rev_by_drama = pos_df[pos_df.title == title].reviews
    reviews_by_drama.append(','.join(rev_by_drama))

In [15]:
len(reviews)

18991

# 필요 함수들

In [ ]:
# pip install contextualized_topic_models

In [ ]:
# pip install tensorflow

In [16]:
import re
import pandas as pd
import numpy as np
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
# from contextualized_topic_models.models.ctm import CombinedTM
# from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
# from konlpy.tag import Okt
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [17]:
from nltk.corpus import stopwords
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
stop_words.update(('\r\n', '\n\r', 'ever', 'much', 'look', 'squid', 'show', 'thing', "i've", 'anything', 'something', "show's",
                   'www', 'soompi' 'com', 'instagram', 'youtube', 'https', 'mydramalist', 'twitter', 'episode', 'comment', 'scene',
                   'version', "he's", 'gonna', 'series', 'watch', 'everything', 'something', "can't", 'list', 'dramas', 'drama',
                   'wait', 'preview', 'someone', 'everyone', 'dont', 'think', 'season', 'anyone', 'something', 'anything', 'nothing', 'world',
                   'status', 'week', 'name', 'cause', 'time', 'en', 'org', 'wikipedia', 'wiki', 'pbs', 'twimg', 'year', 'point', 'please', 'today',
                   'haha', 'case', 'guess', 'reason', 'person', 'moment', 'sense', 'kinda', 'part', 'movie', 'school', 'start', 'work', 'lead', 'kind',
                   'rate', 'rating', 'rate', 'men', 'example', 'idea', 'half', 'review', 'genre', 'side', "that's", "they're", 'till', 'tell', 'phone',
                   'section', 'number', 'company', 'line', "there's", 'male', 'team', 'rating', 'baby', 'course', 'care', 'cute', 'question', 'help', 'group',
                   'hand', 'spoiler', 'hate', 'need', 'mess', 'change', 'drop', 'date', 'netflix', 'yeah', 'daon', 'park', 'thank', 'lmao', 'damn', "i'll",
                   'kang', 'shinwoo', 'taekyung', 'mean', 'woman', 'hope', 'read', 'fact', 'opinion', 'stuff', 'feel', 'kdrama', 'talk', 'song', 'hype',
                   'title', 'type'))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\g8428\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\g8428\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\g8428\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\g8428\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# 사용 함수

In [18]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    elif pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

# CustomTokenizer class setting

In [19]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, a):
        # a = ' '.join(a)
        word_tokens = self.tagger(a)
        
        words = []
        for i in word_tokens:
            text = re.sub('[^a-zA-Z0-9\']','',i).strip()
            text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
            if(text != ''):
                words.append(text)
        
        tag_words = nltk.pos_tag(words)
        pos_words = [word for word in tag_words if word[1][0] in {'N'}] #'V','N','J','R'

        temp_list = []
        for token, pos_tag in pos_words:
            tag = get_wordnet_pos(pos_tag)
            if tag != None:
                temp_list.append((token, get_wordnet_pos(pos_tag)))
        lemma = WordNetLemmatizer()
        token_final = [lemma.lemmatize(token, pos=tag) for token, tag in temp_list]
        long_words = [i for i in token_final if len(i) > 2]
        results = [w for w in long_words if w not in stop_words]
        
        return results

# 작업

In [20]:
def tfidf_vectorizing(reviews):
    ngram_range = (1,2)

    custom_tokenizer = CustomTokenizer(text_to_word_sequence)

    tfidf = TfidfVectorizer(tokenizer=custom_tokenizer,ngram_range = ngram_range,\
        stop_words = stop_words, max_df=10, min_df=2 ,max_features=5000).fit(reviews)
    candidates = tfidf.get_feature_names()
    
    return candidates

# 임베딩

In [21]:
def embedding(sentences,candidates):
    from sentence_transformers import SentenceTransformer

    model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

    doc_embedding = model.encode([sentences])
    candidate_embeddings = model.encode(candidates)
    # drama_embedding = model.encode(reviews_by_drama)

    return doc_embedding, candidate_embeddings

# 코사인 유사도 함수

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
top_n = 500
diversity = 0.7

# 벡터포함 함수

def mmr_with_vec(doc_embedding, candidate_embeddings, words, top_n, diversity):
    word_doc_distances = cosine_similarity(candidate_embeddings,doc_embedding)
    word_distances = cosine_similarity(candidate_embeddings) 

    keywords_idx = [np.argmax(word_doc_distances)]
    candidates_idx = [i for i in range(len(candidates)) if i != keywords_idx[0]]
    
    for _ in range(top_n-1):
        try:
        # 후보 키워드들의 문서유사도 값
            candidate_similarities = word_doc_distances[candidates_idx, :]
            # 후보 키워드와 가장 유사한 키워드
            target_similarities = np.max(word_distances[candidates_idx][:,keywords_idx], axis=1) 

            mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1,1)
            mmr_idx = candidates_idx[np.argmax(mmr)]

            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)
        except:
            break

    # 본 단어 임베딩벡터에서 추출키워드에 해당하는 벡터만
    keywords_vector = candidate_embeddings[keywords_idx][:] 
    keywords = [words[idx] for idx in keywords_idx]
    
    return keywords, keywords_vector


In [25]:
# 벡터미포함 함수
top_n = 10
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
    word_doc_distances = cosine_similarity(candidate_embeddings,doc_embedding)
    word_distances = cosine_similarity(candidate_embeddings) 

    keywords_idx = [np.argmax(word_doc_distances)]
    candidates_idx = [i for i in range(len(candidates)) if i != keywords_idx[0]]
    
    for _ in range(top_n-1):
        try:
        # 후보 키워드들의 문서유사도 값
            candidate_similarities = word_doc_distances[candidates_idx, :]
            # 후보 키워드와 가장 유사한 키워드
            target_similarities = np.max(word_distances[candidates_idx][:,keywords_idx], axis=1) 

            mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1,1)
            mmr_idx = candidates_idx[np.argmax(mmr)]

            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)
        except:
            break
    keywords = [words[idx] for idx in keywords_idx]
    
    return keywords

In [31]:
# for idx, drama in enumerate(drama_embedding):
#     keywords, keywords_vector = mmr(drama.reshape(1,-1) ,candidate_embeddings,candidates,top_n,diversity)
#     print(titles[idx])
#     print(keywords[:10],'\n')

### keyword Dataframe 생성

In [43]:
import pickle

# drama titles in csv
drama_titles = df.groupby('title',as_index=False).count().title

output = pd.DataFrame(None,columns=drama_titles)

top_n = 7
diversity = 0.6

for i,title in tqdm(enumerate(drama_titles)):
    reviews = neg_df[neg_df.title == title].reviews.to_list()

    # tfidf custom vectorizing
    candidates = tfidf_vectorizing(reviews) # reviews = list

    # embedding
    sentences = ','.join(reviews)
    doc_embedding, candidate_embeddings = embedding(sentences, candidates) 

    # keywords extraction

    keywords = mmr(doc_embedding,candidate_embeddings,candidates,top_n,diversity)

    if len(keywords) < top_n:
        less = top_n - len(keywords)
        for i in range(less):
            keywords.append(None)
    output[title] = keywords
    print(f"'{title}': {keywords},")

# output.to_csv('0506_pos_Keywords.csv')

1it [00:12, 12.59s/it]

'18어게인': ['family wife', 'jung', 'stigma', 'zac', 'idol', 'news anchor', 'stalker'],


2it [00:24, 12.32s/it]

'365:운명을거스르는1년': ['suspense', 'kdramas', 'eps', 'romance day', 'creator', 'hyung', 'killer'],


3it [00:37, 12.44s/it]

...

'한번다녀왔습니다': ['romance', 'noona', 'jerk', 'owner', 'writer', 'sister hee', 'law'],


91it [20:25, 12.84s/it]

'해피니스': ['zombie plot', 'patience', 'pandemic', 'chaos happiness', 'character actor', 'couple chemistry', 'claudiakdrama'],


92it [20:41, 13.49s/it]

'호텔델루나': ['romance dismay', 'flashback memory', 'manager handsome', 'acting plot', 'goo chemistry', 'hong sister', 'tree immortality'],


In [44]:
output.to_csv('0509_neg_keywords.csv')

In [32]:
a = {"18어게인" : ['pain stunning', 'dohyun actor', 'daughter daughter', 'people plot', "love episode's", 'flower evil', 'hour screentime'],
"365:운명을거스르는1년" : ['character suspense', 'hyun joon', 'man chemistry', 'sweet loophole', 'universe travel', 'soundtrack', 'comedy'],
"60일지정생존자" : ['awkward romance', 'survivor day', 'ost cinematography', 'jin hee', 'sigh popularity', 'president politics', 'home run'],
"D.P." : ['korea admire', 'abuse bullying', 'cinemaescapist com', 'prison playbook', 'husband', 'hae rain', 'character chemistry'],
'검색어를입력하세요WWW' : ['power romance', 'job sunbae', 'goal mistake', 'clumsily life', 'kdramas female', 'actor jae', 'shoe life'],
'무브투헤븐:나는유품정리사입니다': ["asperger's character", 'love story', 'cage fighting', 'character development', 'jin hee', 'death lot', 'sang uncle'],
'보좌관:세상을움직이는사람들' : ['corruption politics', 'tae jun', 'love', 'character motivation', 'kdramas', 'house card', 'staff deal'],
'갯마을차차차' : ['love people', 'recipe disaster', 'pacing actress', 'day beach', 'horror thriller', 'character introspection', 'cha fishing'],
'괴물' : ['wise evil', "couple's chemistry", 'baeksang performance', 'pacing actor', 'character mare', 'butcher shop', 'cinematography sizzling'],
'구경이' : ['excitement mystery', 'female perpetrator', 'kdramas', 'soundtrack context', 'rest cast', 'cat mouse', 'psychopath'],
'그녀의사생활' : ['favorite story', "jae wook's", 'soundtrack rewatch', 'shower', 'mother mother', 'misunderstanding couple', 'character development'],
'그림자미녀' : ['character story', 'victory guy', 'lamb yang', 'darker beauty', 'online friend', 'chemistry', 'mom'],
'기름진멜로' : ['romance food', 'jung ryeo', 'kdramas', 'gangster', 'jealousy', 'writer', 'sweetness hour'],
'김비서가왜그럴까' : ['chemistry insane', 'trailer judge', 'surgery actress', 'mix cheer', 'storytelling', 'dull', 'romance draggy'],
'나빌레라' :['poetry compassionate', 'deokchul ballet', 'job lot', 'dream family', 'cinematography misstep', "korea's history", 'heartwarming'],
'나의아저씨' :['relationship feeling', 'poverty office', 'intelligent', 'pain life', 'politics way', 'acting rest', 'cast acting'],
'낭만닥터김사부' :['fan doctor', 'yoon reum', 'life lesson', 'character problem', 'ost cinematography', 'bit romance', 'rest cast'],
'내뒤에테리우스' :['charm', 'action comedy', 'kim bon', 'parent', 'soundtrack', 'air spy', 'kdramas'],
'녹두꽃' :['favorite', 'jeon bong', 'people battle', 'character plot', 'forestella lyric', 'sympathy', 'revolution country'],
'단하나의사랑': ['favorite plot', 'kim dan', 'angel pray', 'directing', 'ost love', 'reaper', 'sun acting'],
'대박부동산': ['glad plot', 'actress', 'daebak realty', 'people paranormal', 'badass', 'chemistry cast', 'television wit'],
'도시남녀의사랑법': ['love jae', 'camera thief', 'acting cast', 'korean life', 'chemistry actor', 'pixie dream', 'writer flaw'],
'동백꽃필무렵' :['outcast dongbaek', 'love love', 'mother mother', 'com thriller', 'baseball star', 'meal', 'sadness'],
'라이브': ['character importance', 'wife jang', 'roller coaster', 'soo actor', 'bit romance', 'pacing', 'police corruption'],
'라이프온마스' :['theory dream', 'manicure murder', 'busan', 'vibe', 'kdramas crime', 'mar bowie', 'chemistry'],
'라켓소년단': ['heartwarming', 'korea badminton', 'prison playbook', 'kdramas romance', 'acting', 'friend friend', 'sport development'],
'런온': ['caring sister', 'twist convention', 'film translator', 'father thinking', 'pace feeling', 'hospital playlist', 'character study'],
'로맨스는별책부록' :['love publishing', 'mother divorcee', 'character dan', 'plot twist', 'homeless', 'romcom', 'life lesson'],
'로스쿨' :['law jargon', 'sister', 'character chemistry', 'mastermind', 'goosebump han', 'rest', "bum's acting"],
'마녀식당으로오세요' :['witch wish', 'actor', 'hyeop bullying', 'cook diner', 'iqiyi taiwan', 'quick premise', 'myvideo'],
'마우스' :['killer mouse', 'rest cast', 'rookie detective', 'emotion psychopath', 'goosebump', 'anime', 'mania festival'],
'멜로가체질' :['fun product', 'therapist', 'story ost', 'story couple', 'rewatch', 'comedy challenge', 'pilot'],
'모범택시': ['taxi revenge', 'versatility actor', 'bullying cast', 'rainbow squad', 'vigilante law', 'twin struggle', 'farm'],
'미스터션샤인' :['happiness tragedy', 'butcher', 'fight joseon', 'performance changing', 'history cinematography', 'sister', 'bitter soundtrack'],
'미스티' :['wook love', 'news anchor', 'murder thriller', 'studio bouquet', 'actress baeksang', 'car sens', 'martyr handsome'],
'미치지않고서야' :['insanity', 'director lesson', 'love', 'applause job', 'ban seok', 'actor', 'decision trick'],
'백일의낭군님' :['story joy', 'actor chemistry', 'music rewatch', 'kyungsoo actor', 'weed filler', 'cast lot', 'model day'],
'본대로말하라' :['fan crime', 'jang hyuk', 'actress', 'action man', 'pacing', 'helpful', 'couple revelation'],
'부부의세계': ['suspense', 'wife sun', 'actor husband', 'woo tae', 'hospital', 'slice life', 'sex violence'],
'뷰티인사이드' :['power actress', 'dream priest', 'com dramafoxblog', 'plot twist', 'love chemistry', 'suffers prosopagnosia', 'focus chracters'],
'비밀의숲' :['stranger misogyny', 'television teamwork', 'repeat essay', 'perfect policewoman', "simok yeojin's", 'prison playbook', 'character chemistry'],
'사내맞선': ['perfection jealous', "business proposal's", 'zombie crime', 'girl twist', 'manhwa manhwa', 'ost rewatch', 'punch interviewer'],
'사랑의불시착': ['love cast', 'accident korea', 'sadness anger', 'story businesswoman', 'philosopher tomato', 'legend sea', 'music rewatch'],
'사의찬미': ['feeling love', 'story ost', 'actress', 'joseon', 'knife', 'cheating', 'cinematography'],
'사이코지만괜찮아' :['character heartache', 'kim moon', 'tea purest', 'beauty gang', 'enlistment comeback', 'hyun caregiver', 'murder butterfly'],
'서른아홉' :['character lot', "jo's", 'dermatologist clinic', 'romance couple', 'hangover', 'download thirty', 'onlykdrama com'],
'서른이지만열일곱입니다':['character charm', 'cry', 'soundtrack ost', 'pacing story', 'romcoms twist', 'sun woo', 'heartwarming end'],
'새빛남고학생회' :['woo crush', 'chef', "watching afraid'", 'dildo', 'desk light', 'story story', 'bully tae'],
'소년심판' :['storytelling cinematography', 'kim hyesoo', 'juvenile delinquency', 'society lot', 'punishment belief', 'success', 'pacing'],
'손theguest' :['bloodcurdling demon', 'cinematography quality', 'seo yoon', 'character plot', 'creepy', 'romance chemistry', 'lacklustre acting'],
'술꾼도시여자들' :['drinking culture', 'wife plenty', 'plot', 'eun husband', 'yoga instructor', 'joy sorrow', 'personality'],
'스물다섯스물하나': ['dream relationship', 'chemistry taeri', 'rainstorm tumblr', 'hindsight', 'heedo dad', 'twenty eps', 'evolution character'],
'스카이캐슬' :['parent lot', 'university korea', 'masterpiece acting', 'anxiety', 'prison playbook', 'cup tea', 'people plot'],
'스토브리그' :["drama' depth", 'day baseball', 'sport anime', 'business', 'novice pity', 'politics baseball', 'prison playbook'],
'슬기로운의사생활': ['mind story', "jun's sister", 'mediocre friendship', 'cardio thoracic', 'love alarm', 'cast acting', 'hwa food'],
'시를잊은그대에게' :['comedy', 'poem day', 'lot people', 'romance', 'doctor end', 'dae bang', 'slice life'],
'시맨틱에러' :['cuteness overload', 'ghost doctor', 'chemistry butterfly', 'unassuming plot', 'sangwoo trash', 'adaption manhwa', 'jaeyoung match'],
'아는와이프' :['character mistake', 'wife seo', 'chemistry cast', 'perfection', 'people choice', 'moral story', 'jin life'],
'악의꽃' :['mind thriller', 'village mother', 'cake acting', 'couple chemistry', 'scriptwriter', 'war arrow', 'evil flower'],
'악의마음을읽는자들' :["korea's profiler", 'suspense crime', 'cup tea', 'adaptation book', 'psychology criminal', 'fiery priest', 'hunter'],
'안녕나야' :['hyun lot', 'friend family', 'growth chef', 'bullying', 'bit plot', 'star life', 'routine'],
'어느날우리집현관으로멸망이들어왔다': ['shy cast', 'coffee', 'foxylittlethings', 'death learning', 'tumor heat', "aunt's", 'story fantasy'],
'어쩌다발견한하루' :['plot favorite', 'stage kyung', 'parent dan', 'rewatch', 'manga awareness', 'actress lot', 'story kdramas'],
'오월의청춘': ['love hyun', 'uprising pain', 'memoir', 'actor skill', 'university student', 'cinematography chemistry', 'democracy activist'],
'오징어게임' :["'squid game'", 'dubbing actor', 'royale korea', 'plenty blood', 'masterpiece parasite', 'chemistry manga', 'cookie challenge'],
'옷소매붉은끝동' :['intrigue romance', 'kdramas king', 'politics history', 'scriptwriter jung', 'book memoir', "i'm rewatch", 'power life'],
'왓쳐':['thriller fan', 'seo joon', 'hospital', 'cast', 'attack', 'story tragedy', 'wife'],
'왕이된남자' :['politics', 'strength weakness', 'kdramas romance', 'production storyline', 'taste', 'actress', 'life interest'],
'우리들의블루스' :['people problem', 'actor slice', 'jeju', 'love story', 'chemistry', 'vibe', 'actress'],
'우수무당가두심' :['fantasy', 'hae seon', 'classmate', 'shaman', 'acting cast', 'ghost hyun', 'binge'],
'우아한친구들' :['people slice', 'kim hee', 'rest', 'murder mystery', 'castle sky', 'doctor', 'fight'],
'원더우먼' :['sang yoon', 'revenge', 'chemistry', 'sister law', 'opera plot', 'comedy', 'fiery priest'],
'월간집' :['romance comedy', 'day', 'jung', 'parasite', 'rest cast', 'romcom plot', 'camellia bloom'],
'유미의세포들' :['story romance', 'day actor', "animation yumi's", 'worry hunger', 'goblin', 'flaw people', 'star cell'],
'으라차차와이키키' :['humor romance', 'production', 'zombie', 'rewatch', 'actress', 'dream', 'excitement'],
'의사요한': ['love doctor', 'punch', 'sung character', 'pain death', 'actor lot', 'politics patient', 'mother'],
'이구역의미친X': ['mad story', 'therapy', 'seo actress', 'boyfriend', 'taste', 'comedy comedy', 'cop anger'],
'이리와안아줘': ['masterful plot', 'jang doubt', 'actress', 'character chemistry', 'cinematography', 'nature story', 'cry'],
'인간수업': ['plot twist', 'eat dog', 'hyun actress', 'pressure parent', 'fan crime', 'fight brawl', 'class acting'],
'자백' :['perfection actress', 'chemistry father', 'rewatch', 'story cast', 'hyun', 'corruption', 'romance'],
'작은신의아이들' :['plot', 'translation', 'character', 'bin', 'crime', 'turn', 'writing'],
'지옥' :['hellbound thriller', 'train writer', 'cgi misstep', 'harry potter', 'value people', 'film director', 'opium mass'],
'카이로스' :['thriller romance', 'kdramas', 'character plan', 'perfectly future', 'wife', 'cinematography', 'seo jin'],
'킹덤' :['history joseon', 'addition thriller', 'fan actress', 'zombie train', 'court intrigue', 'nature morality', 'cgi makeup'],
'태종이방원': ["horse's", 'korean', 'entertainment', 'abuse', 'wife', 'dragon', 'fiction'],
'트랩' :['thriller', 'trap', 'plot', None, None, None, None],
'트레이서' :['tracer', 'mbc', 'storyline', 'taxpayersubteam', 'pirate google', 'translator', 'fan'],
'하늘에서내리는일억개의별' :['thriller mystery', "kdrama's", 'actor korea', 'parent story', 'commenter faint', 'love love', 'jung chemistry'],
'하이에나': ['romance comedy', 'hyena giriboy', 'parameter reality', 'soo actress', 'badass', 'skill', 'thriller ost'],
'한번다녀왔습니다': ['favourite family', 'jung eun', 'rollercoaster', 'acting cast', 'sunbae', 'fighting', 'story story'],
'해피니스' :['happiness zombie', 'problem couple', 'cure vaccine', 'character acting', 'chemistry villain', 'thriller suspense', 'hindrance happiness'],
'호텔델루나': ['cinematography beautiful', 'odyssey sister', "kim's favorite", 'prison playbook', 'death resentment', 'concept hotel', 'fan plot']}

In [33]:
pos_keys = pd.DataFrame(a)

In [37]:
pos_keys.to_csv('0509_pos_keywords.csv')

In [ ]:
# with open('title.pickle', 'rb') as f:
#     keywords = pickle.load(f)
#     keywords_vector = pickle.load(f)